# Shohei Ohtani Pitching Stats

- 大谷翔平の投手成績を可視化する例
- サンプルデータはkm/h & m 単位になっていますが, [オリジナルデータ](https://baseballsavant.mlb.com/statcast_search)はmile/h & feetなのでご注意ください

In [61]:
import pandas as pd

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [62]:
# データ読み込み. game_dateは予めデータ型を変更

df = pd.read_csv('./dataset/20221005_baseball-savant-shohei-ohtani-pitch.csv', header=0)

df['game_date'] = pd.to_datetime(df['game_date'])
df.head(10)

,game_date,game_month,game_pk,inning,pitcher_name,p_throws,batter_name,stand,at_bat_number,pitch_number,pitch_name,effective_speed,release_spin_rate,release_extension,type,events,description,des,zone,bb_type,launch_speed_angle,hit_location,launch_speed,launch_angle,hit_distance_sc,pfx_x,pfx_z,plate_x,plate_z,release_pos_x,release_pos_z,effective_speed_kh,release_extension_cm,pfx_x_cm,pfx_z_cm,plate_x_cm,plate_z_cm,release_pos_x_cm,release_pos_z_cm,launch_speed_angle_category,launch_speed_kh,hit_distance_sc_m,hc_x,hc_y,hc_x_0,hc_y_0,hc_x_ft,hc_y_ft,hc_x_0_ft,hc_y_0_ft,hc_x_m,hc_y_m,hc_x_0_m,hc_y_0_m,events_summary
0,2022-10-05,10,662421,1.0,"Ohtani, Shohei",R,"Kemp, Tony",L,6,1,Cutter,90.3,2242.0,6.7,S,NaN,called_strike,Tony Kemp called out on strikes.,3.0,NaN,NaN,NaN,NaN,NaN,NaN,0.12,0.65,0.46,2.97,-2.37,5.83,144.48,204.22,3.66,19.81,14.02,90.53,-72.24,177.70,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
1,2022-10-05,10,662421,1.0,"Ohtani, Shohei",R,"Kemp, Tony",L,6,2,Sinker,97.6,1882.0,6.8,S,NaN,foul,Tony Kemp called out on strikes.,4.0,NaN,NaN,NaN,75.1,36.0,174.0,-1.40,0.97,-0.32,2.61,-2.32,5.84,156.16,207.26,-42.67,29.57,-9.75,79.55,-70.71,178.00,Unknown,120.16,53.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
2,2022-10-05,10,662421,1.0,"Ohtani, Shohei",R,"Kemp, Tony",L,6,3,Split-Finger,86.7,1179.0,6.8,B,NaN,ball,Tony Kemp called out on strikes.,14.0,NaN,NaN,NaN,NaN,NaN,NaN,-0.31,-0.02,1.52,0.46,-1.77,6.26,138.72,207.26,-9.45,-0.61,46.33,14.02,-53.95,190.80,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
3,2022-10-05,10,662421,1.0,"Ohtani, Shohei",R,"Kemp, Tony",L,6,4,Cutter,87.6,2343.0,6.7,B,NaN,ball,Tony Kemp called out on strikes.,14.0,NaN,NaN,NaN,NaN,NaN,NaN,0.44,-0.27,0.36,0.59,-1.99,6.26,140.16,204.22,13.41,-8.23,10.97,17.98,-60.66,190.80,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
4,2022-10-05,10,662421,1.0,"Ohtani, Shohei",R,"Kemp, Tony",L,6,5,Slider,86.3,2365.0,6.6,S,strikeout,called_strike,Tony Kemp called out on strikes.,7.0,NaN,NaN,NaN,NaN,NaN,NaN,0.79,0.73,-0.32,1.53,-2.68,5.72,138.08,201.17,24.08,22.25,-9.75,46.63,-81.69,174.35,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,strikeout
5,2022-10-05,10,662421,1.0,"Ohtani, Shohei",R,"Murphy, Sean",R,7,1,Cutter,91.0,2425.0,6.7,S,NaN,swinging_strike,Sean Murphy flies out to center fielder Mike T...,14.0,NaN,NaN,NaN,NaN,NaN,NaN,0.21,0.34,1.17,2.11,-2.39,5.76,145.60,204.22,6.40,10.36,35.66,64.31,-72.85,175.56,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
6,2022-10-05,10,662421,1.0,"Ohtani, Shohei",R,"Murphy, Sean",R,7,2,Cutter,92.9,2403.0,6.7,B,NaN,ball,Sean Murphy flies out to center fielder Mike T...,14.0,NaN,NaN,NaN,NaN,NaN,NaN,0.16,0.29,0.94,1.74,-2.57,5.72,148.64,204.22,4.88,8.84,28.65,53.04,-78.33,174.35,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
7,2022-10-05,10,662421,1.0,"Ohtani, Shohei",R,"Murphy, Sean",R,7,3,Sinker,98.6,2007.0,7.0,S,NaN,called_strike,Sean Murphy flies out to center fielder Mike T...,8.0,NaN,NaN,NaN,NaN,NaN,NaN,-1.16,0.38,-0.27,1.80,-2.27,5.80,157.76,213.36,-35.36,11.58,-8.23,54.86,-69.19,176.78,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
8,2022-10-05,10,662421,1.0,"Ohtani, Shohei",R,"Murphy, Sean",R,7,4,Slider,84.1,2418.0,6.6,X,field_out,hit_into_play,Sean Murphy flies out to center fielder Mike T...,9.0,fly_ball,3.0,NaN,80.8,39.0,273.0,0.40,-0.57,0.66,2.08,-1.80,6.38,134.56,201.17,12.19,-17.37,20.12,63.40,-54.86,194.46,Under,129.28,83.21,107.46,91.33,-17.96,106.94,268.65,228.32,-44.9,267.35,81.88,69.59,-13.69,81.49,other
9,2022-10-05,10,662421,1.0,"Ohtani, Shohei",R,"Brown, Seth",L,8,1,Curveball,72.9,2330.0,6.7,B,NaN,ball,"Seth Brown grounds out, third baseman Luis Ren...",14.0,NaN,NaN,NaN,NaN,NaN,NaN,0.81,-1.23,2.02,0.87,-1.80,6.40,116.64,204.22,24.69,-37.49,61.57,26.52,-54.86,195.07,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other


In [63]:
# 念の為データ型を確認
df.dtypes

game_date                      datetime64[ns]
game_month                              int64
game_pk                                 int64
inning                                float64
pitcher_name                           object
p_throws                               object
batter_name                            object
stand                                  object
at_bat_number                           int64
pitch_number                            int64
pitch_name                             object
effective_speed                       float64
release_spin_rate                     float64
release_extension                     float64
type                                   object
events                                 object
description                            object
des                                    object
zone                                  float64
bb_type                                object
launch_speed_angle                    float64
hit_location                      

## Color Map定義

- 球種, イベントに合わせてカラーマップを定義
- 色は基本的に[オリジナルデータ](https://baseballsavant.mlb.com/statcast_search)の掲載元に合わせて定義
- Plotlyで描画するときに引数として渡す


In [64]:
from typing import Dict
class Color:
    EVENTS: Dict[str, str] = {
        'catcher_interf': 'rgb(255, 255, 255)',
        'caught_stealing_2b': 'rgb(255, 255, 255)',
        'caught_stealing_3b': 'rgb(255, 255, 255)',
        'caught_stealing_home': 'rgb(255, 255, 255)',
        'double': 'rgb(119, 94, 240)',
        'double_play': 'rgb(194, 194, 194)',
        'field_error': 'rgb(255, 255, 255)',
        'field_out': 'rgb(194, 194, 194)',
        'fielders_choice': 'rgb(255, 255, 255)',
        'fielders_choice_out': 'rgb(255, 255, 255)',
        'force_out': 'rgb(194, 194, 194)',
        'game_advisory': 'rgb(255, 255, 255)',
        'grounded_into_double_play': 'rgb(194, 194, 194)',
        'hit_by_pitch': 'rgb(255, 255, 255)',
        'home_run': 'rgb(214, 43, 126)',
        'other_out': 'rgb(194, 194, 194)',
        'pickoff_1b': 'rgb(255, 255, 255)',
        'pickoff_2b': 'rgb(255, 255, 255)',
        'pickoff_3b': 'rgb(255, 255, 255)',
        'pickoff_caught_stealing_3b': 'rgb(255, 255, 255)',
        'pickoff_caught_stealing_home': 'rgb(255, 255, 255)',
        'sac_bunt': 'rgb(255, 255, 255)',
        'sac_fly': 'rgb(255, 255, 255)',
        'sac_fly_double_play': 'rgb(194, 194, 194)',
        'single': 'rgb(253, 97, 0)',
        'strikeout': 'rgb(194, 194, 194)',
        'strikeout_double_play': 'rgb(194, 194, 194)',
        'triple': 'rgb(255, 177, 2)',
        'triple_play': 'rgb(194, 194, 194)',
        'walk': 'rgb(255, 255, 255)',
        'wild_pitch': 'rgb(255, 255, 255)',
    }

    BB_TYPE: Dict[str, str] = {
        'fly_ball': 'rgb(80, 95, 58)',
        'ground_ball': 'rgb(183, 21, 131)',
        'line_drive': 'rgb(236, 236, 76)',
        'popup': 'rgb(222, 30, 36)',
    }

    LAUNCH_SPEED_ANGLE: Dict[str, str] = {
        'Weak': 'rgb(255, 255, 255)',
        'Topped': 'rgb(194, 194, 194)',
        'Under': 'rgb(253, 97, 0)',
        'Flare/Burner': 'rgb(119, 94, 240)',
        'Solid Contact': 'rgb(255, 177, 2)',
        'Barrel': 'rgb(214, 43, 126)',
    }

    PITCH_TYPE: Dict[str, str] = {
        '2-Seam Fastball': 'rgb(254, 157, 0)',
        '4-Seam Fastball': 'rgb(210, 46, 73)',
        'Changeup': 'rgb(28, 191, 58)',
        'Curveball': 'rgb(0, 210, 236)',
        'Cutter': 'rgb(148, 63, 44)',
        'Eephus': 'rgb(255, 255, 255)',
        'Fastball': 'rgb(210, 46, 73)',
        'Knuckle Curve': 'rgb(98, 54, 204)',
        'Knuckleball': 'rgb(0, 0, 0)',
        'Sinker': 'rgb(254, 157, 0)',
        'Slider': 'rgb(237, 231, 22)',
        'Split-Finger': 'rgb(58, 172, 172)',
    }

    DESCRIPTION: Dict[str, str] = {
        'ball': 'rgb(32, 119, 180)',
        'blocked_ball': 'rgb(32, 119, 180)',
        'bunt_foul_tip': 'rgb(42, 160, 45)',
        'called_strike': 'rgb(254, 127, 11)',
        'foul': 'rgb(42, 160, 45)',
        'foul_bunt': 'rgb(42, 160, 45)',
        'foul_tip': 'rgb(42, 160, 45)',
        'hit_by_pitch': 'rgb(0, 0, 0)',
        'hit_into_play': 'rgb(149, 102, 188)',
        'missed_bunt': 'rgb(255, 255, 255)',
        'pitchout': 'rgb(255, 255, 255)',
        'swinging_strike': 'rgb(227, 120, 193)',
        'swinging_strike_blocked': 'rgb(227, 120, 193)',
    }


## 投球データの可視化

- どこに投げられているか
- 球種毎, 結果毎にプロット

In [65]:
# グラフ描画用の関数

import plotly.express as px
import plotly.graph_objects as go

# ゾーンの横幅と縦幅
PLATE_ZONE_X = (-100, 100)
PLATE_ZONE_Y = (0, 225)

# ストライクゾーン定義（本当は上下可変ですがわかりやすいように固定にしています）
PLATE_STRIKE_ZONE_X = [-28.9, -28.9, 28.9, 28.9, -28.9]
PLATE_STRIKE_ZONE_Y = [48, 104, 104, 48, 48]

HOVER = [
    "game_date",
    "batter_name",
    "pitcher_name",
    "effective_speed_kh",
    "pitch_name",
    "hit_distance_sc_m",
    "launch_speed_kh",
    "launch_angle",
    "des",
    "description"
]
LEGEND = dict(
    yanchor="top",
    y=1,
    xanchor="left",
    x=0,
    orientation="h"
)
def strike_zone(name: str = 'strike zone') -> go.Scatter:
    return go.Scatter(
        name=name,
        x=PLATE_STRIKE_ZONE_X,
        y=PLATE_STRIKE_ZONE_Y,
        line_color='red'
    )


def plate(df: pd.DataFrame, title: str = 'Pitch on plate(Umpire view)',
            hover: str = 'pitch_name', colormap: Dict[str, str] = Color.PITCH_TYPE) -> go.Figure:
    fig = px.scatter(
        df.sort_values(hover),
        x="plate_x_cm",
        y="plate_z_cm",
        color=hover,
        color_discrete_map=colormap,
        hover_name=hover,
        hover_data=HOVER
    )
    fig.add_trace(strike_zone())
    fig.update_layout(
        title=title,
        autosize=False,
        width=800,
        height=800,
        legend=LEGEND,
        legend_title=None
    )
    fig.update_xaxes(range=PLATE_ZONE_X, title='plate_x(cm)', dtick=25)
    fig.update_yaxes(range=PLATE_ZONE_Y, title='plate_x(cm)', dtick=25)
    return fig


In [66]:
# 早速可視化してみる

_fig = plate(df)
_fig.show()

In [67]:
# 結果ごとに可視化

_fig = plate(df, hover='events', colormap=Color.EVENTS)
_fig.show()

In [68]:
# 一試合に絞る. 9/17, 7回無失点の試合

_fig = plate(df[df['game_date'] == '2022-09-17'])
_fig.show()

## リリースした位置

- どこからリリースしたか, 高さと横位置を可視化
- キャッチャー（審判）から見た視点での可視化
  - TV中継みたいに投手の背中越しに見たい場合はX軸を逆転させると可能

In [69]:
def release(df: pd.DataFrame, title: str = 'Release Point View(Umpire view)',
            hover: str = 'pitch_name', colormap: Dict[str, str] = Color.PITCH_TYPE) -> go.Figure:
    fig = px.scatter(
        df.sort_values(hover),
        x="release_pos_x_cm",
        y="release_pos_z_cm",
        color=hover,
        color_discrete_map=colormap,
        hover_name=hover,
        hover_data=HOVER
    )
    fig.add_trace(strike_zone())
    fig.update_layout(
        title=title,
        autosize=False,
        width=800,
        height=800,
        legend=LEGEND,
        legend_title=None
    )
    fig.update_xaxes(range=(-125, 125), title='release_pos_x(cm)', dtick=25)
    fig.update_yaxes(range=(0, 275), title='release_pos_z(cm)', dtick=25)
    return fig


In [70]:
_fig = release(df[df['game_date'] == '2022-09-17'])
_fig.show()

# 投げたボールの比率

変化球の種類を円グラフで表す

In [71]:
def pie(df: pd.DataFrame, key: str, title: str) -> go.Figure:
    df_copy: pd.DataFrame = df.copy()
    df_copy['count'] = 1
    # Key毎にサマって数える
    df_pie = df_copy[[key, 'count']].groupby(key).count().reset_index().sort_values(key)
    fig = px.pie(
        df_pie,
        values='count',
        names=key,
        color=key,
        color_discrete_map=Color.PITCH_TYPE
    )
    fig.update_layout(
        title=title,
        autosize=True,
        legend_title=None
    )
    return fig


In [72]:
_fig = pie(df, key='pitch_name', title='球種割合')
_fig.show()

## 投球データを使った時系列分析

- どの球種を何球投げているか（Pitch Line）
  - X: 球数（右に行けば行くほど最新）
  - Y: 球速
- 試合日ごとの球種と球速の分布
  - X: 試合日
  - Y: 球速

In [73]:
from typing import List
# 球種ごとに数えるため, ソートし直してのち番号を振る
df_copy = df.sort_values(['game_date', 'game_pk', 'inning', 'at_bat_number', 'pitch_number']).copy()
pitch_names: List[str] = df_copy['pitch_name'].unique().tolist()
list_df: List[pd.DataFrame] = []
for pitch_name in pitch_names:
    _df_p = df_copy[df_copy['pitch_name'] == pitch_name]
    _df_p['pitches'] = range(1, len(_df_p.index) + 1)
    list_df.append(_df_p)
df_timeline: pd.DataFrame = pd.concat(list_df, axis=0)


/var/folders/xg/msdwc4cj1csdv205nm1cdn7w0000gn/T/ipykernel_62699/2593106426.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/xg/msdwc4cj1csdv205nm1cdn7w0000gn/T/ipykernel_62699/2593106426.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/xg/msdwc4cj1csdv205nm1cdn7w0000gn/T/ipykernel_62699/2593106426.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [74]:
# グラフ描画の関数
BALLS = [0, 1200]
RANGE_SPEED = [60, 180]

def pitch_line(df: pd.DataFrame,
                title: str = 'Pitch Line(effective speed & ball count)', hover: str = 'pitch_name') -> go.Figure:
    fig = px.scatter(
        df.sort_values(hover),
        x="pitches",
        y="effective_speed_kh",
        color=hover,
        color_discrete_map=Color.PITCH_TYPE,
        hover_name=hover,
        hover_data=HOVER
    )
    fig.update_layout(
        title=title,
        autosize=False,
        width=1200,
        height=800,
        legend=LEGEND,
        legend_title=None
    )
    fig.update_xaxes(range=BALLS, title='Pitches', dtick=100)
    fig.update_yaxes(range=RANGE_SPEED, title='effective_speed_kh', dtick=20)
    return fig

def time_line(df: pd.DataFrame,
                title: str = 'Timeline(effective speed & Game Date)', hover: str = 'pitch_name') -> go.Figure:
    fig = px.scatter(
        df.sort_values(hover),
        x="game_date",
        y="effective_speed_kh",
        color=hover,
        color_discrete_map=Color.PITCH_TYPE,
        hover_name=hover,
        hover_data=HOVER
    )
    fig.update_layout(
        title=title,
        autosize=False,
        width=1200,
        height=800,
        legend=LEGEND,
        legend_title=None
    )
    fig.update_xaxes(title='Game Date')
    fig.update_yaxes(range=RANGE_SPEED, title='effective_speed_kh', dtick=20)
    return fig


In [75]:
# Pitch Line
_fig = pitch_line(df_timeline)
_fig.show()

In [76]:
# Timeline
_fig = time_line(df_timeline)
_fig.show()